In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sklearn

In [5]:
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, MaxPooling1D, Conv1D, Dropout
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input, TimeDistributed, LSTM, Bidirectional

In [7]:
y_train = np.loadtxt('/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Data/vid_features_BoVW_openFace_210_Pose_Gaze_AUs/y_train_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',')
y_dev = np.loadtxt('/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Data/vid_features_BoVW_openFace_210_Pose_Gaze_AUs/y_dev_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',')

In [8]:
x_train = np.loadtxt('/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Data/vid_features_BoVW_openFace_210_Pose_Gaze_AUs/x_train_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',').reshape(163, 19663, 101)
x_dev = np.loadtxt('/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Data/vid_features_BoVW_openFace_210_Pose_Gaze_AUs/x_dev_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',').reshape(56, 36277, 101)

x_train_input_shape = x_train.shape
x_dev_input_shape = x_dev.shape

zeros = np.zeros((163, x_dev_input_shape[1]-x_train_input_shape[1], x_train_input_shape[2]))

# adding rows with value 0 to make the dimensions of x_train and x_dev equal
x_train = np.concatenate((x_train, zeros), axis=1)

x_train_input_shape = x_train.shape
x_dev_input_shape = x_dev.shape

print(x_train_input_shape)
print(x_dev_input_shape)

(163, 36277, 101)
(56, 36277, 101)


In [9]:
model = Sequential()

"""
    Default values of Batch Normalization are:
        Momentum defaults to 0.99
        The hyperparameter ε defaults to 0.001
        The hyperparameter β defaults to an all-zeros vector
        The hyperparameter γ defaults to an all-ones vector
"""

model.add(Conv1D(16, (3), padding='same', activation='relu', input_shape=(x_train_input_shape[1], x_train_input_shape[2])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(32, (3), padding='same', activation='relu', input_shape=(x_train_input_shape[1], x_train_input_shape[2])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(64, (3), padding='same', activation='relu', input_shape=(x_train_input_shape[1], x_train_input_shape[2])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(128, (3), padding='same', activation='relu', input_shape=(x_train_input_shape[1], x_train_input_shape[2])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))


model.add(LSTM(32, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))

model.add(LSTM(16, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))


model.add(Flatten())
model.add(Dense(1, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), 
                    bias_regularizer=regularizers.L2(1e-4), 
                    activity_regularizer=regularizers.L2(1e-5), 
                    activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 36277, 16)         4864      
                                                                 
 batch_normalization (BatchN  (None, 36277, 16)        64        
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 36277, 16)         0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 18139, 16)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 18139, 32)         1568      
                                                                 
 batch_normalization_1 (Batc  (None, 18139, 32)       

In [11]:
batch_size = 32

checkpoint_filepath = '/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Model_Checkpoints/BoVW_OpenFace/weights-improvement-BatchNormalization-Dropout-Regularization-{epoch:02d}-{val_accuracy:.2f}.hdf5'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=20,
            callbacks=[model_checkpoint_callback],
            validation_data=(x_dev, y_dev),
            shuffle=True)

Epoch 1/20
6/6 [==============================] - ETA: 0s - loss: 1.2069 - accuracy: 0.6748
Epoch 1: val_accuracy improved from -inf to 0.76786, saving model to /Users/nitanshjain/Documents/Miscellaneous/Depression_Analysis/Model_Checkpoints/BoVW_OpenFace/weights-improvement-BatchNormalization-Dropout-Regularization-01-0.77.hdf5
6/6 [==============================] - 55s 8s/step - loss: 1.2069 - accuracy: 0.6748 - val_loss: 0.6832 - val_accuracy: 0.7679
Epoch 2/20
6/6 [==============================] - ETA: 0s - loss: 1.3058 - accuracy: 0.6564
Epoch 2: val_accuracy did not improve from 0.76786
6/6 [==============================] - 39s 6s/step - loss: 1.3058 - accuracy: 0.6564 - val_loss: 0.6244 - val_accuracy: 0.7679
Epoch 3/20
6/6 [==============================] - ETA: 0s - loss: 1.0087 - accuracy: 0.7669
Epoch 3: val_accuracy did not improve from 0.76786
6/6 [==============================] - 33s 5s/step - loss: 1.0087 - accuracy: 0.7669 - val_loss: 0.7149 - val_accuracy: 0.6071
Ep

# Test Data

In [ ]:
# x_test = np.loadtxt('/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Data/vid_features_BoVW_openFace_210_Pose_Gaze_AUs/x_test_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',').reshape(56, 0, 101)
# y_test = np.loadtxt('/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Data/vid_features_BoVW_openFace_210_Pose_Gaze_AUs/y_test_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',')

# x_test_input_shape = x_test.shape
# y_test_input_shape = y_test.shape

# zeros = np.zeros((56, x_dev_input_shape[1]-x_test_input_shape[1], x_test_input_shape[2]))

# # adding rows with value 0 to make the dimensions of x_test and x_dev equal
# x_test = np.concatenate((x_test, zeros), axis=1)

# x_test_input_shape = x_test.shape

# print(x_test_input_shape)
# print(y_test_input_shape)

In [ ]:
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

# Models Accuracy

```
Conv1D + MaxPooling1D + 2 LSTM Layers + Flatten + Dense + Adam Optimizer
```
Epoch 12/20
6/6 [==============================] - 477s 80s/step - loss: 0.4566 - accuracy: 0.7975 - val_loss: 0.7239 - val_accuracy: 0.7857